In [1]:
import sqlite3
import csv
import os

In [2]:
conn = sqlite3.connect('balt_trends.db')
cursor = conn.cursor()

In [3]:


with open('resources/balt_cleaned.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)
    insert_data_query = '''
    INSERT INTO balt_trends (OBJECTID, CCNO, CrimeDateTime, CrimeCode, Location, Description, Post, Gender, Age, Race, Ethnicity, District, Neighborhood, Latitude, Longitude, Total_Incidents)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
    for row in csvreader:
        cursor.execute(insert_data_query, row)
        conn.commit()

In [4]:
cursor.execute('SELECT * FROM balt_trends')
rows = cursor.fetchall()

In [6]:
import pandas as pd
balt_crime = pd.DataFrame(rows, columns =[col[0] for col in cursor.description])

In [7]:
balt_crime.head()

,OBJECTID,CCNO,CrimeDateTime,CrimeCode,Location,Description,Post,Gender,Age,Race,Ethnicity,District,Neighborhood,Latitude,Longitude,Total_Incidents
0,1,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,833,F,15,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.2733,-76.6924,1
1,2,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,833,F,15,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.2733,-76.6924,1
2,3,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,833,F,27,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.2733,-76.6924,1
3,4,23F08231,2023/06/24 04:01:00+00,3JK,600 LUCIA AVE,ROBBERY,833,M,25,BLACK_OR_AFRICAN_AMERICAN,UNKNOWN,SOUTHWEST,YALE HEIGHTS,39.2733,-76.6924,1
4,5,23F08235,2023/06/24 03:45:00+00,5A,3200 LILY AVE,BURGLARY,922,M,48,nan,HISPANIC_OR_LATINO,SOUTHERN,CHERRY HILL,39.2464,-76.6368,1


In [8]:
num_nan_per_row = balt_crime.isnull().sum(axis=1)

# Count the total number of rows with at least one NaN value
num_rows_with_nan = num_nan_per_row.sum()

print("Number of rows with NaN values:", num_rows_with_nan)


Number of rows with NaN values: 0


In [9]:
balt_crime = balt_crime.drop(['Post', 'Location', 'Gender', 'Age', 'Race', 'Ethnicity', 'Latitude', 'Longitude', 'OBJECTID', 'CCNO'], axis = 1)
balt_crime

,CrimeDateTime,CrimeCode,Description,District,Neighborhood,Total_Incidents
0,2023/06/24 04:01:00+00,4B,AGG. ASSAULT,SOUTHWEST,YALE HEIGHTS,1
1,2023/06/24 04:01:00+00,4B,AGG. ASSAULT,SOUTHWEST,YALE HEIGHTS,1
2,2023/06/24 04:01:00+00,4B,AGG. ASSAULT,SOUTHWEST,YALE HEIGHTS,1
3,2023/06/24 04:01:00+00,3JK,ROBBERY,SOUTHWEST,YALE HEIGHTS,1
4,2023/06/24 03:45:00+00,5A,BURGLARY,SOUTHERN,CHERRY HILL,1
...,...,...,...,...,...,...
1705837,2011/03/01 22:11:00+00,4D,AGG. ASSAULT,NORTHERN,WAVERLY,1
1705838,2011/03/01 20:30:00+00,6J,LARCENY,SOUTHERN,WESTPORT,1
1705839,2011/03/01 20:00:00+00,7A,AUTO THEFT,SOUTHWEST,EDMONDSON VILLAGE,1
1705840,2011/03/01 14:40:00+00,4B,AGG. ASSAULT,WESTERN,SANDTOWN-WINCHESTER,1


In [10]:
from datetime import datetime
import pytz
datetime_string = "2023/06/24 04:01:00+00"


datetime_string_no_tz = datetime_string[:-3]
dt_object = datetime.strptime(datetime_string_no_tz, "%Y/%m/%d %H:%M:%S")
timezone_utc = pytz.timezone('UTC')
dt_object = timezone_utc.localize(dt_object)
day = dt_object.day
month = dt_object.month
year = dt_object.year
hour = dt_object.hour
day_of_week = dt_object.strftime('%A')

balt_crime['Day'] = day
balt_crime['Month'] = month
balt_crime['Year'] = year
balt_crime['Hour'] = hour
balt_crime['DOW'] = day_of_week
balt_crime.head()

,CrimeDateTime,CrimeCode,Description,District,Neighborhood,Total_Incidents,Day,Month,Year,Hour,DOW
0,2023/06/24 04:01:00+00,4B,AGG. ASSAULT,SOUTHWEST,YALE HEIGHTS,1,24,6,2023,4,Saturday
1,2023/06/24 04:01:00+00,4B,AGG. ASSAULT,SOUTHWEST,YALE HEIGHTS,1,24,6,2023,4,Saturday
2,2023/06/24 04:01:00+00,4B,AGG. ASSAULT,SOUTHWEST,YALE HEIGHTS,1,24,6,2023,4,Saturday
3,2023/06/24 04:01:00+00,3JK,ROBBERY,SOUTHWEST,YALE HEIGHTS,1,24,6,2023,4,Saturday
4,2023/06/24 03:45:00+00,5A,BURGLARY,SOUTHERN,CHERRY HILL,1,24,6,2023,4,Saturday


In [26]:
from sklearn.preprocessing import LabelEncoder

X=balt_crime.drop(columns=['District'])
y=balt_crime['District']

In [27]:
cc_encoder = LabelEncoder()

balt_crime['CrimeCode'] = cc_encoder.fit_transform(balt_crime["CrimeCode"])

In [44]:
desc_encoder = LabelEncoder()

balt_crime['Description'] =desc_encoder.fit_transform(balt_crime["Description"])

In [43]:
dist_encoder = LabelEncoder()
dist_name = dist_encoder.fit_transform(y)
balt_crime['District'] =dist_encoder.fit_transform(balt_crime["District"])

In [45]:
neighborhood_encoder = LabelEncoder()

balt_crime['Neighborhood'] =neighborhood_encoder.fit_transform(balt_crime["Neighborhood"])

In [46]:
year_encoder = LabelEncoder()

balt_crime['Year'] =year_encoder.fit_transform(balt_crime["Year"])

In [47]:
dow_encoder = LabelEncoder()

balt_crime['DOW'] =dow_encoder.fit_transform(balt_crime["DOW"])

In [48]:
balt_crime

,CrimeDateTime,CrimeCode,Description,District,Neighborhood,Total_Incidents,Day,Month,Year,Hour,DOW
0,2023/06/24 04:01:00+00,44,0,7,276,1,24,6,0,4,0
1,2023/06/24 04:01:00+00,44,0,7,276,1,24,6,0,4,0
2,2023/06/24 04:01:00+00,44,0,7,276,1,24,6,0,4,0
3,2023/06/24 04:01:00+00,30,9,7,276,1,24,6,0,4,0
4,2023/06/24 03:45:00+00,48,3,6,45,1,24,6,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...
1705837,2011/03/01 22:11:00+00,46,0,3,257,1,24,6,0,4,0
1705838,2011/03/01 20:30:00+00,63,6,6,263,1,24,6,0,4,0
1705839,2011/03/01 20:00:00+00,66,2,7,73,1,24,6,0,4,0
1705840,2011/03/01 14:40:00+00,44,0,8,229,1,24,6,0,4,0


In [11]:
from numpy.random.mtrand import sample
sample_data = balt_crime.sample(frac = 0.25, random_state=32)
sample_data

,CrimeDateTime,CrimeCode,Description,District,Neighborhood,Total_Incidents,Day,Month,Year,Hour,DOW
403663,2014/05/07 10:00:00+00,6E,LARCENY,NORTHEAST,BELAIR-EDISON,1,24,6,2023,4,Saturday
447696,2013/05/26 18:32:00+00,6B,LARCENY,SOUTHEAST,PATTERSON PARK NEIGHBORHOOD,1,24,6,2023,4,Saturday
895501,2016/01/10 02:00:00+00,5A,BURGLARY,NORTHWEST,CROSS COUNTRY,1,24,6,2023,4,Saturday
252677,2017/06/30 20:00:00+00,7A,AUTO THEFT,SOUTHERN,CURTIS BAY,1,24,6,2023,4,Saturday
337516,2015/11/03 15:00:00+00,5A,BURGLARY,SOUTHERN,WASHINGTON VILLAGE/PIGTOWN,1,24,6,2023,4,Saturday
...,...,...,...,...,...,...,...,...,...,...,...
367078,2015/02/24 11:00:00+00,4E,COMMON ASSAULT,SOUTHEAST,HOPKINS BAYVIEW,1,24,6,2023,4,Saturday
1314889,2018/12/20 04:00:00+00,5E,BURGLARY,SOUTHEAST,GRACELAND PARK,1,24,6,2023,4,Saturday
233087,2017/11/04 10:00:00+00,6D,LARCENY FROM AUTO,NORTHERN,ROLAND PARK,1,24,6,2023,4,Saturday
786055,2018/04/12 12:18:00+00,6J,LARCENY,NORTHEAST,BELAIR-EDISON,1,24,6,2023,4,Saturday


In [35]:
from sklearn.preprocessing import LabelEncoder

X=sample_data.drop(columns=['District'])
y=sample_data['District']

In [49]:
desc_encoder = LabelEncoder()
desc_numeric = desc_encoder.fit_transform(y)
sample_data['Description'] =desc_encoder.fit_transform(sample_data['Description'])


In [50]:
cc_encoder = LabelEncoder()
z_numeric = cc_encoder.fit_transform(y)
sample_data['CrimeCode'] =cc_encoder.fit_transform(sample_data["CrimeCode"])


In [51]:
dist_encoder = LabelEncoder()
dist_name = dist_encoder.fit_transform(y)
sample_data['District'] =dist_encoder.fit_transform(sample_data["District"])


In [52]:
neighborhood_encoder = LabelEncoder()
y_numeric = neighborhood_encoder.fit_transform(y)
sample_data['Neighborhood'] =neighborhood_encoder.fit_transform(sample_data["Neighborhood"])


In [53]:
year_encoder = LabelEncoder()
sample_data['Year'] =year_encoder.fit_transform(sample_data["Year"])


In [54]:
dow_encoder = LabelEncoder()
sample_data['DOW'] =dow_encoder.fit_transform(sample_data["DOW"])

In [55]:
sample_data

,CrimeDateTime,CrimeCode,Description,District,Neighborhood,Total_Incidents,Day,Month,Year,Hour,DOW
403663,2014/05/07 10:00:00+00,59,6,2,11,1,24,6,0,4,0
447696,2013/05/26 18:32:00+00,56,6,5,196,1,24,6,0,4,0
895501,2016/01/10 02:00:00+00,48,3,4,53,1,24,6,0,4,0
252677,2017/06/30 20:00:00+00,66,2,6,55,1,24,6,0,4,0
337516,2015/11/03 15:00:00+00,48,3,6,256,1,24,6,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...
367078,2015/02/24 11:00:00+00,47,4,5,119,1,24,6,0,4,0
1314889,2018/12/20 04:00:00+00,52,3,5,96,1,24,6,0,4,0
233087,2017/11/04 10:00:00+00,58,7,3,219,1,24,6,0,4,0
786055,2018/04/12 12:18:00+00,63,6,2,11,1,24,6,0,4,0


In [56]:
from sklearn.model_selection import train_test_split
X = sample_data[['CrimeCode', 'Neighborhood', 'Day', 'Month', 'Year', 'Hour', 'DOW', 'Description', 'Total_Incidents']]
y = sample_data['District']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [57]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=0)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [58]:
predictions = rf_model.predict(X_test)

In [59]:
accuracy = rf_model.score(X_test, y_test)
print('Prediction accuracy:', accuracy)

Prediction accuracy: 0.9618018102518407


In [60]:
print(balt_crime['District'])

0          7
1          7
2          7
3          7
4          6
          ..
1705837    3
1705838    6
1705839    7
1705840    8
1705841    5
Name: District, Length: 1705842, dtype: int32


In [61]:
dist_encoder.inverse_transform(predictions)
selected_district = y_test[predictions ==1].iloc[0]
print('Best District for a new business:', selected_district)

Best District for a new business: 1


In [62]:
predicted_hours = dist_encoder.inverse_transform(predictions)

In [63]:
decoded_districts = dist_encoder.inverse_transform(balt_crime['District'])

In [64]:
predictions_df = pd.DataFrame(predictions, columns=['Predicted_District'])
y_test_df = pd.DataFrame(y_test.values, columns=['Actual_District'])


In [65]:
result_df = pd.concat([y_test_df, predictions_df], axis=1)
print(result_df)

       Actual_District  Predicted_District
0                    1                   1
1                    0                   0
2                    0                   0
3                    5                   5
4                    1                   1
...                ...                 ...
85287                1                   1
85288                0                   0
85289                2                   2
85290                1                   1
85291                0                   0

[85292 rows x 2 columns]


In [ ]:
folder_path = r'C:\Users\mitch\crime-trends\resources'

In [ ]:
conn = sqlite3.connect(folder_path + 'balt_trends.db')

In [ ]:
conn.close()